# OpenAI

## Basic example

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Summarize this text..."}
    ]
)

print(response.choices[0].message.content)


The text discusses the key factors to consider when choosing a vacation rental property. These factors include location, budget, amenities, reviews, and rules/policies. It is important to carefully research and evaluate different rental options before making a decision.


## Get key words from prompt

In [26]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Charger la clé API depuis .env
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Objectif de recherche du client
research_objective = """
At present, one of our clients is looking to speak with professionals who have insights about the emerging technologies 
in soft contact lens manufacturing, particularly non-injection moulded methods. They would broadly like to understand how 
these technologies are reshaping the industry—from on-demand manufacturing to smart, drug-delivery-enabled lenses.
"""

# Prompt pour GPT spécifiant OpenAlex et PatentView
prompt = f"""
You are helping a researcher generate search keywords.

Return a JSON object with:
- "openalex": a list of academic search terms
- "patentview": a list of patent keywords
- "cpc_codes": a list of CPC classification codes

Rules:
- Use valid JSON only
- No markdown, no comments, no explanations, no inline comments

Research Objective:
\"\"\"
{research_objective}
\"\"\"
"""




# Appel à l’API OpenAI
response = client.chat.completions.create(
    #model="gpt-3.5-turbo",
    model="gpt-4-turbo",
    messages=[{"role": "user", "content": prompt}]
)

# Affichage du résultat
print(response.choices[0].message.content)


{
  "openalex": [
    "soft contact lenses",
    "contact lens manufacturing",
    "non-injection molding contact lens",
    "on-demand lens manufacturing",
    "smart contact lenses",
    "drug delivery contact lenses",
    "advanced contact lens technology"
  ],
  "patentview": [
    "non-injection molding",
    "contact lens technology",
    "custom contact lenses",
    "smart lenses",
    "ophthalmic lens",
    "drug dispensing contact lenses",
    "flexible contact lens manufacturing"
  ],
  "cpc_codes": [
    "G02C7/04",
    "G02B1/04",
    "B29D11/00",
    "A61F2/14",
    "A61F9/00",
    "A61L27/00",
    "B29L11/00"
  ]
}


In [28]:
import json

response_text = response.choices[0].message.content.strip()
data = json.loads(response_text)

openalex_keywords = data["openalex"]
patentview_keywords = data["patentview"]
cpc_codes = data["cpc_codes"]

print(openalex_keywords)


['soft contact lenses', 'contact lens manufacturing', 'non-injection molding contact lens', 'on-demand lens manufacturing', 'smart contact lenses', 'drug delivery contact lenses', 'advanced contact lens technology']


# Open alex

## Openalex basic example

In [26]:
import requests
import pandas as pd

# ==== CONFIGURATION ====
query = "contact lens manufacturing"
per_page = 15
mailto = "adyl.elguamra@gmail.com"   # Replace with your email to join polite pool
api_key = ""  # Optional: replace with your key if you have one

# ==== API REQUEST ====
url = "https://api.openalex.org/works"
params = {
    "search": query,
    "per-page": per_page,
    "mailto": mailto
}

# Include API key if available
if api_key:
    params["api_key"] = api_key

response = requests.get(url, params=params)

# ==== HANDLE RESPONSE ====
if response.status_code == 200:
    data = response.json()
    works = data.get("results", [])
    
    # Extract info into list of dicts
    records = []
    for work in works:
        record = {
            "Title": work.get("title"),
            "Year": work.get("publication_year"),
            "Citations": work.get("cited_by_count"),
            "Authors": ", ".join([auth["author"]["display_name"] for auth in work.get("authorships", [])]),
            "OpenAlex ID": work.get("id"),
        }
        records.append(record)
    
    # Convert to DataFrame
    df = pd.DataFrame(records)
    display(df)

else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)


,Title,Year,Citations,Authors,OpenAlex ID
0,Daily Wear Contact Lenses Manufactured in Etaf...,2016,13,"Loretta Szczotka‐Flynn, Sara M. Debanne, Beth ...",https://openalex.org/W2533308847
1,Possible hazard to contact lens manufacture.,1973,5,Rosalyn Routledge,https://openalex.org/W2029290835
2,Revolutionizing contact lens manufacturing: ex...,2024,9,"Lina M. Shaker, Ahmed A. Al‐Amiery, Wan Nor Ro...",https://openalex.org/W4392398549
3,Contact Lens Manufacturing,2018,3,"Stephen D. Newman, John Clamp",https://openalex.org/W2905185084
4,Automation of Pick and Place Operation in Cont...,2018,3,"Norhaliza Abdul Wahab, Jeevaniswaran Thangaraj",https://openalex.org/W2891397204
5,A novel application of UV-LEDs in the contact ...,2005,3,"Sharon McDermott, James E. Walsh, Robert Howard",https://openalex.org/W2000037419
6,Biomedical applications of hydrogels: A review...,2014,2285,"Enrica Caló, Vitaliy V. Khutoryanskiy",https://openalex.org/W2044512869
7,Prospects for Additive Manufacturing in Contac...,2020,44,"Fahad Alam, Mohamed Elsherif, Bader AlQattan, ...",https://openalex.org/W3083896593
8,Impact of manufacturing technology and materia...,2005,33,"Carole Maldonado‐Codina, Nathan Efron",https://openalex.org/W2093492547
9,Impact of manufacturing technology and materia...,2004,33,"Carole Maldonado‐Codina, Nathan Efron",https://openalex.org/W2113155567


## Openalex with abstract

In [3]:
import requests
import pandas as pd

# ==== CONFIGURATION ====
query = "contact lens 3d printing"
per_page = 15
mailto = "adyl.elguamra@gmail.com"   # Your email for OpenAlex contact header
api_key = ""  # Optional API key if available

# ==== FUNCTION TO RECONSTRUCT ABSTRACT ====
def reconstruct_abstract(indexed_abstract):
    if not indexed_abstract:
        return ""
    position_map = {}
    for word, positions in indexed_abstract.items():
        for pos in positions:
            position_map[pos] = word
    abstract = " ".join([position_map[i] for i in sorted(position_map)])
    return abstract

# ==== API REQUEST ====
url = "https://api.openalex.org/works"
params = {
    "search": query,
    "per-page": per_page,
    "mailto": mailto,
    "filter": "publication_year:2010-2025",
    "sort": "publication_year:desc"
}


if api_key:
    params["api_key"] = api_key

response = requests.get(url, params=params)

# ==== PROCESS RESPONSE ====
if response.status_code == 200:
    data = response.json()
    works = data.get("results", [])
    
    records = []
    for work in works:
        abstract_index = work.get("abstract_inverted_index")
        abstract = reconstruct_abstract(abstract_index)

        # Get all institutions for all authors
        institutions = []
        for auth in work.get("authorships", []):
            for inst in auth.get("institutions", []):
                name = inst.get("display_name")
                if name:
                    institutions.append(name)

        record = {
            "Title": work.get("title"),
            "Year": work.get("publication_year"),
            "Citations": work.get("cited_by_count"),
            "Authors": ", ".join([auth["author"]["display_name"] for auth in work.get("authorships", [])]),
            "Institutions": ", ".join(sorted(set(institutions))),  # Remove duplicates
            "Abstract": abstract,
            "OpenAlex ID": work.get("id"),
        }
        records.append(record)
    
    # Convert to DataFrame
    df = pd.DataFrame(records)
    print(df)

else:
    print(f"Request failed. Status code: {response.status_code}")
    print(response.text)


                                                Title  Year  Citations  \
0   Most Important Biomedical and Pharmaceutical A...  2025          0   
1   Three-Dimensional Printing for Accessible and ...  2025          0   
2   Biomaterials in Relative Devices for Traumatic...  2025          0   
3   Accelerated customization of contact lenses th...  2025          0   
4   Vis-OCT imaging-driven 3D printing of customiz...  2025          0   
5   3D-Printed Contact Lenses to Release Polyvinyl...  2025          0   
6   Myelin ensheathment and drug responses of olig...  2025          0   
7   Unveiling the complex morphologies of sessile ...  2025          1   
8   Clinical evaluation of corneal ulcer with a po...  2025          1   
9   Synthesis of PVA-Based Hydrogels for Biomedica...  2025          3   
10  A Simple Technique for Studying the Interactio...  2025          0   
11  An In-Vitro Evaluation of Strength, Hardness, ...  2025          0   
12  Metal Additive Manufacturing and M

## Openalex with similarity

In [ ]:
import requests
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# ==== CONFIGURATION ====
prompt = "contact lens 3d printing"
keywords = openalex_keywords
per_page = 100
max_results = 500
mailto = "adyl.elguamra@gmail.com"
api_key = ""  # Optional

# ==== MODEL SETUP ====
model = SentenceTransformer('all-MiniLM-L6-v2')
prompt_embedding = model.encode(prompt, convert_to_tensor=True)

# ==== ABSTRACT RECONSTRUCTION FUNCTION ====
def reconstruct_abstract(indexed_abstract):
    if not indexed_abstract:
        return ""
    position_map = {}
    for word, positions in indexed_abstract.items():
        for pos in positions:
            position_map[pos] = word
    abstract = " ".join([position_map[i] for i in sorted(position_map)])
    return abstract

# ==== INITIALIZE SEARCH ====
url = "https://api.openalex.org/works"
search_query = " OR ".join(f'"{kw}"' for kw in keywords)
cursor = "*"
results_fetched = 0
scored_records = []

# ==== PAGINATED FETCH ====
while results_fetched < max_results:
    params = {
        "search": search_query,
        "per-page": per_page,
        "mailto": mailto,
        "filter": "publication_year:2010-2025",
        "sort": "publication_year:desc",
        "cursor": cursor
    }
    if api_key:
        params["api_key"] = api_key

    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Request failed. Status code: {response.status_code}")
        print(response.text)
        break

    data = response.json()
    works = data.get("results", [])
    cursor = data.get("meta", {}).get("next_cursor", None)
    if not cursor:
        break  # No more pages

    for work in works:
        title = work.get("title", "")
        abstract_index = work.get("abstract_inverted_index")
        abstract = reconstruct_abstract(abstract_index)

        # Combine title and abstract for similarity
        text_to_compare = f"{title}. {abstract}" if abstract else title
        text_embedding = model.encode(text_to_compare, convert_to_tensor=True)
        similarity = util.cos_sim(text_embedding, prompt_embedding).item()

        institutions = []
        for auth in work.get("authorships", []):
            for inst in auth.get("institutions", []):
                name = inst.get("display_name")
                if name:
                    institutions.append(name)

        scored_records.append({
            "Title": title,
            "Year": work.get("publication_year"),
            "Citations": work.get("cited_by_count"),
            "Authors": ", ".join([auth["author"]["display_name"] for auth in work.get("authorships", [])]),
            "Institutions": ", ".join(sorted(set(institutions))),
            "Abstract": abstract,
            "OpenAlex ID": work.get("id"),
            "Similarity": similarity
        })

        results_fetched += 1
        if results_fetched >= max_results:
            break

# ==== TO DATAFRAME ====
df = pd.DataFrame(sorted(scored_records, key=lambda x: x["Similarity"], reverse=True))
print(df[["Title", "Year", "Citations", "Similarity"]])


                                                Title  Year  Citations  \
0   Healthcare with smart contact lenses: innovati...  2025          1   
1   Smart Contact Lenses: Disease Monitoring and T...  2025          0   
2     Orthokeratology-A Historical Summary and Update  2025          0   
3   A perspective on Smart contact lenses: Pioneer...  2025          1   
4   Power Profiles of Different Myopia Control Sof...  2025          0   
..                                                ...   ...        ...   
95  Advancements in Passive Wireless Sensing Syste...  2025          4   
96  Influence of next-generation artificial intell...  2025          2   
97  Advances in wearable energy storage and harves...  2025          9   
98  Wearable sensor-based real time monitoring sys...  2025          0   
99  Unveiling the nanoworld of antimicrobial resis...  2025          1   

    Similarity  
0     0.601612  
1     0.554421  
2     0.527253  
3     0.522083  
4     0.518932  
..       

# Serpapi Google Patents AP

https://serpapi.com/blog/scraping-google-patents-with-python-and-serpapi/ 

In [29]:
# Import required libraries
from dotenv import load_dotenv
import pandas as pd
import os
from serpapi import GoogleSearch

# Load .env variables
load_dotenv()
api_key = os.getenv("SERPAPI_KEY")

# Search parameters
params = {
    "engine": "google_patents",
    # "q": "3d printing contact lens",  # Your search term
    "q":'"3D printing" AND ("contact lens" OR "hydrogel lens")',
    "api_key": api_key,
    "num": 10,                        # Number of results on this page (max 100)
    "hl": "en",
    "gl": "us",
    "after": "20220101"
}

# Perform the search
search = GoogleSearch(params)
results = search.get_dict()

# Extract organic results (patents list)
patent_results = results.get("organic_results", [])
print(f"Found {len(patent_results)} patents on page 1")

# Extract key data
extracted_data = []
for result in patent_results:
    data = {
        "title": result.get("title"),
        "snippet": result.get("snippet"),
        "link": result.get("patent_link"),
        "publication_date": result.get("publication_date"),
        "inventor": result.get("inventor"),
        "assignee": result.get("assignee"),
        "patent_id": result.get("patent_id")
    }
    extracted_data.append(data)

# Save to CSV
df = pd.DataFrame(extracted_data)
df.to_csv("patents_page1.csv", index=False, encoding="utf-8")
print("Saved one-page result to patents_page1.csv")


Found 10 patents on page 1
Saved one-page result to patents_page1.csv


In [30]:
df.head(10)

,title,snippet,link,publication_date,inventor,assignee,patent_id
0,Three-dimensional (3d) printing of electro-act...,1 . A method of three-dimensional (3D) printin...,https://patents.google.com/patent/US2022032418...,2022-10-13,Anthony Van Heugten,"E-Vision Smart Optics, Inc.",patent/US20220324188A1/en
1,Three-dimensional printing of optical devices,". 51 . The method of claim 40 , wherein the de...",https://patents.google.com/patent/US2022027431...,2022-09-01,Ture Kindt-Larsen,"Atheneum Optical Sciences, Llc",patent/US20220274317A1/en
2,Medical devices including medicaments and meth...,What is claimed is: 1. A packaged medical devi...,https://patents.google.com/patent/US11510869B2/en,2022-11-29,Praful Doshi,Praful Doshi,patent/US11510869B2/en
3,Contact hearing device and retention structure...,… structure maintains a location of the transd...,https://patents.google.com/patent/US11979718B2/en,2024-05-07,Peter DY,Earlens Corporation,patent/US11979718B2/en
4,Validation of Authenticity of Association of a...,Techniques to validate authenticity of associa...,https://patents.google.com/patent/US2022036606...,2022-11-17,Nova Spivack,"Blocktag, Inc.",patent/US20220366061A1/en
5,Functionalized and crosslinked polymers,Polyhydric polymers may be converted to deriva...,https://patents.google.com/patent/US2023031276...,2023-10-05,David Gravett,"Pmidg, Llc",patent/US20230312764A1/en
6,Anti-myopia-progression spectacles and associa...,… the optical path length from the center to t...,https://patents.google.com/patent/US12050368B2/en,2024-07-30,Yan Zhou,"Reopia Optics, Inc.",patent/US12050368B2/en
7,Technologies for content analysis,Various computing technologies for content ana...,https://patents.google.com/patent/US2024041255...,2024-12-12,Roman Tsibulevskiy,Patnotate Llc,patent/US20240412550A1/en
8,Negative pressure wound closure device,The present invention relates to a negative pr...,https://patents.google.com/patent/US2023030183...,2023-09-28,Philip David Canner,"Smith &amp; Nephew, Inc.",patent/US20230301838A1/en
9,"Apparatus, systems, and methods for vision ass...",Methods and systems for assessing and/or treat...,https://patents.google.com/patent/US2024015634...,2024-05-16,Tuan Tran,"Vivid Vision, Inc.",patent/US20240156340A1/en


In [ ]:
import os
from dotenv import load_dotenv
from serpapi import GoogleSearch

# Load variables from .env file
load_dotenv()

# Read the API key
api_key = os.getenv("SERPAPI_KEY")

print("API KEY:", api_key)


# Read the API key
api_key = os.getenv("SERPAPI_KEY")

params = {
    "engine": "google_patents",
    "q": "3d printing contact lens",   # Your search term
    "api_key": api_key,
    "num": 10,                       # Number of results
    "hl": "en",                     # Language
    "gl": "us",                     # Country
    "after": "20220101"            # Filter by date (optional)
}

# Send request
search = GoogleSearch(params)
results = search.get_dict()

# Print results
for patent in results.get("patent_results", []):
    print(f"Title: {patent.get('title')}")
    print(f"Link: {patent.get('link')}")
    print(f"Summary: {patent.get('summary')}")
    print(f"Publication Date: {patent.get('publication_date')}")
    print(f"Inventors: {patent.get('inventors')}")
    print("-" * 40)


API KEY: 0b5e8887cd4dd73f635c202b3b7a4fa293f15257ee08101726c5cb7352285293


In [10]:
print(results)

{'search_metadata': {'id': '6836ab8a10c26d34d92bc871', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/b10ad728d43c5af6/6836ab8a10c26d34d92bc871.json', 'created_at': '2025-05-28 06:22:02 UTC', 'processed_at': '2025-05-28 06:22:02 UTC', 'google_patents_url': 'https://patents.google.com/xhr/query?url=q%3Dblockchain%2Bhealthcare%26after%3D20220101%26num%3D10', 'raw_html_file': 'https://serpapi.com/searches/b10ad728d43c5af6/6836ab8a10c26d34d92bc871.html', 'prettify_html_file': 'https://serpapi.com/searches/b10ad728d43c5af6/6836ab8a10c26d34d92bc871.prettify', 'total_time_taken': 4.31}, 'search_parameters': {'engine': 'google_patents', 'q': 'blockchain healthcare', 'after': '20220101', 'num': '10'}, 'search_information': {'total_results': 9373, 'total_pages': 100, 'page_number': 1}, 'organic_results': [{'position': 1, 'rank': 0, 'patent_id': 'patent/KR20240166435A/en', 'patent_link': 'https://patents.google.com/patent/KR20240166435A/en', 'serpapi_link': 'https://serpapi.c

# PatentView
https://patentsview.org/apis/purpose
The current version of the new PatentSearch API delivers data on patents granted through March 31, 2025.

https://github.com/PatentsView/PatentsView-Code-Examples/tree/main

https://github.com/PatentsView/PatentsView-Code-Examples/blob/main/data-downloads/0-getting-started.ipynb

In [ ]:
import os
import time
import requests
from dotenv import load_dotenv

# Load API key
load_dotenv()
API_KEY = os.getenv("PATENTSVIEW_API_KEY")

url = "https://search.patentsview.org/api/v1/patent"
headers = {
    "X-Api-Key": API_KEY,
    "Content-Type": "application/json"
}

# Set to store seen patent IDs (to prevent duplicates)
seen_patents = set()

# Query with broader fields + optional date filter
query_template = {
    "q": {
        "_and": [
            {"_gte": {"patent_date": "2018-01-01"}},
            {"_text_all": {
                "patent_title": "contact lens manufacturing",
                "patent_abstract": "contact lens manufacturing"
            }}
        ]
    },
    "f": [
        "patent_id",
        "patent_title",
        "patent_date",
        "patent_abstract",
        "inventors.inventor_name_first",
        "inventors.inventor_name_last",
        "assignees.assignee_organization",
        "assignees.assignee_city",
        "assignees.assignee_country"
    ],
    "s": [{"patent_date": "desc"}]
}

# Pagination settings
per_page = 10
total_pages = 1

for page in range(1, total_pages + 1):
    print(f"\n📄 Page {page}\n" + "=" * 40)
    
    query = query_template.copy()
    query["o"] = {"per_page": per_page, "page": page}
    
    response = requests.post(url, json=query, headers=headers)

    if response.status_code == 200:
        data = response.json()
        patents = data.get("patents", [])
        if not patents:
            print("No more results.")
            break

        for patent in patents:
            pid = patent["patent_id"]
            if pid in seen_patents:
                continue  # Skip duplicates
            seen_patents.add(pid)

            print(f"\nPatent ID: {pid}")
            print(f"Title: {patent['patent_title']}")
            print(f"Date: {patent['patent_date']}")
            print(f"Abstract: {patent.get('patent_abstract', 'N/A')}")

            # Inventors
            inventors = patent.get("inventors", [])
            if inventors:
                print("Inventors:")
                for inv in inventors:
                    first = inv.get("inventor_name_first", "")
                    last = inv.get("inventor_name_last", "")
                    print(f"  - {first} {last}")
            else:
                print("Inventors: N/A")

            # Assignees
            assignees = patent.get("assignees", [])
            if assignees:
                print("Assignees:")
                for a in assignees:
                    org = a.get("assignee_organization", "Unknown")
                    city = a.get("assignee_city", "")
                    country = a.get("assignee_country", "")
                    print(f"  - {org} ({city}, {country})")
            else:
                print("Assignees: N/A")

        time.sleep(1)  # Respect rate limits
    else:
        print(f"❌ Error on page {page}: {response.status_code}")
        print("Response:", response.text)
        break



📄 Page 1

Patent ID: 12240949
Title: Material for manufacturing contact lenses and contact lens thereof
Date: 2025-03-04
Abstract: A material for manufacturing contact lenses comprises a first composition, a second composition, at least one hydrophilic monomer and at least one photoinitiator. The first composition and the second composition are respectively represented by the following formula (1) and formula (2).Z1 is a first silicone hydrogel monomer, n1 is ranged between 1 and 1.5, and o1 is ranged between 1 and 2; and Z2 is a second silicone hydrogel monomer, n2 is ranged between 1 and 5, and o2 is ranged between 1 and 2.
Inventors:
  - Ji-Yu Lin
  - Yu-Ya Liao
Assignees:
  - INNO VISION CORP. (Taipei, TW)

Patent ID: 12169329
Title: Specialty contact lens design and manufacturing
Date: 2024-12-17
Abstract: Specialty contact lenses can be designed using approximations of one or both surfaces using mathematical representations. The approximations can be configured to optimize the l

# FDA api

In [32]:
import requests
import urllib.parse

# Base API and search query
base_url = "https://api.fda.gov/device/510k.json"
query = 'device_name:"contact lens" AND decision_date:[1990-01-01 TO 2025-01-01]'
encoded_query = urllib.parse.quote(query)
full_url = f"{base_url}?search={encoded_query}&limit=50"

# Fetch and print results sorted by decision date (descending)
response = requests.get(full_url)
if response.status_code == 200:
    data = response.json()
    results = data.get("results", [])
    
    if not results:
        print("No results found.")
    else:
        # Sort by decision date descending
        results.sort(key=lambda x: x.get("decision_date", ""), reverse=True)

        for item in results:
            print("Device Name:", item.get("device_name", "N/A"))
            print("Applicant:", item.get("applicant", "N/A"))
            print("Decision Date:", item.get("decision_date", "N/A"))
            print("K Number:", item.get("k_number", "N/A"))
            print("Summary:", item.get("decision_description", "N/A"))
            print("-" * 50)
else:
    print("Request failed:", response.status_code)
    print("Response:", response.text)


Device Name: VizionFocus (mififilcon A) Silicone Hydrogel Soft (hydrophilic) Daily Disposable Contact Lens; VizionFocus Color (mififilcon A) Silicone Hydrogel Soft (hydrophilic) Daily Disposable Contact Lens
Applicant: VizionFocus Inc.
Decision Date: 2024-12-17
K Number: K242413
Summary: Substantially Equivalent
--------------------------------------------------
Device Name: Flexlens Silicone Hydrogel Daily Wear Soft (Hydrophilic) Contact Lens (efrofilcon A)
Applicant: X-Cel Specialty Contacts, A Walman Optical Company
Decision Date: 2024-08-14
K Number: K242059
Summary: Substantially Equivalent
--------------------------------------------------
Device Name: Acuity 200 (fluoroxyfocon A) Rigid Gas Permeable Contact Lens; Acuity 100 (hexafocon A) Rigid Gas Permeable Contact Lens; Acuity 200 with Tangible® Hydra-PEG® (fluoroxyfocon A) Rigid Gas Permeable Contact Lens; Acuity 100 with Tangible® Hydra-PEG® (hexafocon A) Rigid Gas Permeable Contact Lens
Applicant: Acuity Polymers, Inc.
D